In [1]:
import requests
import os
import math

In [2]:
bill_filenames = []
for file in os.listdir("../ustawy"):
    if file.endswith(".txt"):
        bill_filenames.append(os.path.join("../ustawy", file))

In [3]:
tags = {}
bigrams = {}

for bill_name in bill_filenames:
    with open(bill_name, encoding='utf-8') as b_file:
        content = b_file.read()
        content = content.lower()
        
        res = requests.post('http://localhost:9200', data=content.encode('UTF-8')).text
        
        lines = res.splitlines()
        
        prev = None
        for line in lines:
            line = line.split()
            if len(line) == 3:
                word = line[0]
                tag = line[1].split(':')[0]
                key = (word, tag)
                tags[key] = tags.get(key, 0) + 1
                
                if prev is not None:
                    bigrams[(prev, key)] = bigrams.get((prev, key), 0) + 1
                prev = key
                

In [4]:
def alpha_filter(kv):
    (w1, t1), (w2, t2) = kv[0]
    if w1.isalpha() and w2.isalpha():
        return True
    return False

bigrams = dict(filter(alpha_filter, bigrams.items()))

In [5]:
def H(args, N):
    res = 0
    for k in args:
        k /= N
        k_eq_0 = 1 if k == 0 else 0
        res += k * math.log(k + k_eq_0)
    return res

In [6]:
bigrams_LLR = {}
all_occ = sum(bigrams.values())

In [7]:
for bigram, k11 in bigrams.items():
    (w1, t1), (w2, t2) = bigram
    k12 = tags[(w2, t2)] - k11
    k21 = tags[(w1, t1)] - k11
    
    k22 = all_occ - (k11 + k12 + k21)
    LLR = 2 * all_occ * (H([k11, k12, k21, k22], all_occ) - H([k11 + k12, k21 + k22], all_occ) - H([k11 + k21, k12 + k22], all_occ))
    
    bigrams_LLR[bigram] = LLR   

In [8]:
bigrams_LLR = {k: v for k, v in sorted(bigrams_LLR.items(), key=lambda item: item[1], reverse=True)}

In [9]:
for bigram, llr in list(bigrams_LLR.items())[:10]:
    print(str(bigram) + ': ' + str(llr))

(('który', 'adj'), ('mowa', 'subst')): 248323.47813489943
(('o', 'prep'), ('który', 'adj')): 163662.8380830866
(('mowa', 'subst'), ('w', 'prep')): 149478.78490378824
(('otrzymywać', 'fin'), ('brzmienie', 'subst')): 111105.82963924854
(('minister', 'subst'), ('właściwy', 'adj')): 67763.46033475136
(('w', 'prep'), ('artykuł', 'brev')): 67751.07338376575
(('dodawać', 'fin'), ('się', 'qub')): 66927.24815087883
(('w', 'prep'), ('ustęp', 'brev')): 56401.460472486884
(('stosować', 'fin'), ('się', 'qub')): 52971.0157973974
(('droga', 'subst'), ('rozporządzenie', 'subst')): 51680.8027317483


In [13]:
bigrams_partitioned = {}
for bigram, freq in bigrams.items():
    (w1, t1), (w2, t2) = bigram
    bigrams_partitioned[(t1, t2)] = bigrams_partitioned.get((t1, t2), 0) + freq

In [14]:
bigrams_partitioned = {k: v for k, v in sorted(bigrams_partitioned.items(), key=lambda item: item[1], reverse=True)}

In [15]:
for bigram, freq in list(bigrams_partitioned.items())[:10]:
    print(str(bigram) + ': ' + str(freq))

('prep', 'subst'): 327378
('subst', 'subst'): 290104
('subst', 'adj'): 274761
('adj', 'subst'): 188238
('subst', 'prep'): 173254
('subst', 'conj'): 85145
('conj', 'subst'): 84393
('prep', 'adj'): 79459
('ger', 'subst'): 77510
('prep', 'brev'): 67230


In [36]:
top_LLR_tagged = {}
for t in list(bigrams_partitioned.keys())[:10]:
    top_LLR_tagged[t] = []

for bigram, LLR in bigrams_LLR.items():
    ((w1, t1), (w2, t2)) = bigram
    tags = (t1, t2)
    if tags in top_LLR_tagged.keys():
        if len(top_LLR_tagged[tags]) < 5:
            top_LLR_tagged[tags].append((w1, w2, LLR))

        finished = [len(tags) == 10 for tags in top_LLR_tagged.values()]
        if all(finished):
            break
    


In [37]:
for category, items in top_LLR_tagged.items():
    print(category)
    for item in items:
        print('\t' + str(item))
    print()

('prep', 'subst')
	('na', 'podstawa', 45477.310954122746)
	('do', 'sprawa', 45252.30316708258)
	('z', 'dzień', 43159.23405404215)
	('w', 'droga', 31358.799272838856)
	('od', 'dzień', 29493.159003393183)

('subst', 'subst')
	('droga', 'rozporządzenie', 51680.8027317483)
	('skarb', 'państwo', 21817.45586219784)
	('rada', 'minister', 14668.755602848223)
	('terytorium', 'rzeczpospolita', 14111.888097340187)
	('ochrona', 'środowisko', 13839.313197740266)

('subst', 'adj')
	('minister', 'właściwy', 67763.46033475136)
	('rzeczpospolita', 'polski', 42974.13408095582)
	('jednostka', 'organizacyjny', 23994.10394401504)
	('samorząd', 'terytorialny', 23146.147887406274)
	('produkt', 'leczniczy', 21293.761116537702)

('adj', 'subst')
	('który', 'mowa', 248323.47813489943)
	('niniejszy', 'ustawa', 19890.750397923475)
	('następujący', 'zmiana', 17163.801270255233)
	('odrębny', 'przepis', 11789.581385975374)
	('walny', 'zgromadzenie', 9524.82724318318)

('subst', 'prep')
	('mowa', 'w', 149478.78490378

# Podsumowanie
## What types of bigrams have been found?
Znalezione bigramy o najwyższej wartośći LLR to albo takie zawierające przyimki ("o który", "mowa w"), będące czasownikami zwrotnymi lub różne połączenia przymiotników z innymi częściami mowy. 

## Which of the category-pairs indicate valuable multiword expressions? Do they have anything in common?
Największą wartość wnoszą bigramy z kategorii: ('subst', 'adj') - rzeczownik przymiotnik, które określają nazwy własne jak: "minister właściwy", "samorząd terytorialny"

## Which signal: LLR score or syntactic category is more useful for determining genuine multiword expressions?
Nawiązując to poprzedniego pytania uważam, że połączenie obu. Najlepiej wybrać parę takich części mowy, które faktycznie mogą utworzyć coś sensownego, a następnie policzyć LLR. W ten sposób w zależności od tego czy szukamy nazw własnych, jakiejś konkretnej formy, jak czasowniki zwrotne, czy często występujące pary przyimek przymiotnik możemy je znaleźć na podstawie tych o najwyższym LLR w danej kategorii.

## Can you describe a different use-case where the morphosyntactic category is useful for resolving a real-world problem?
Jak już zacząłem powyżej, możemy tego użyć do podpowiadania następnego słowa, na przykład gdy jest to czasownik zwrotny lub nazwa własna.